In [5]:
a = [(1,2,3),(4,5,6),(7,8,9)]
x = [(y,z) for x,y,z in a if x%2]
print([v for x, y, z in a if x % 2 for v in (y, z)])
print([x for *_,x in a])

[2, 3, 8, 9]
[3, 6, 9]


In [ ]:
import asyncio
import ccxt.pro

async def main():
    exchange = ccxt.pro.binance()  # or another supported exchange
    await exchange.load_markets()

    symbol = 'ETC/ETH'

    # Subscribe to live ticker updates via WebSocket
    ticker = await asyncio.wait_for(
                            exchange.watch_order_book(symbol),
                            timeout=10.0
                        )
    
    print(ticker)
    exchange.che
    await exchange.close()

await main()


{'bids': [[0.00476, 26.6], [0.00475, 132.36], [0.00474, 415.09], [0.00473, 8.71], [0.00472, 10.92], [0.00471, 470.39], [0.0047, 10.97], [0.00469, 3.64], [0.00468, 0.62], [0.00467, 4.74], [0.00466, 0.28], [0.00465, 6.84], [0.00464, 0.28], [0.00463, 3.0], [0.00462, 0.34], [0.00461, 5.39], [0.0046, 111.4], [0.00459, 3.3], [0.00458, 0.57], [0.00457, 12.02], [0.00456, 37.89], [0.00455, 3.68], [0.00454, 0.35], [0.00453, 3.64], [0.00452, 1010.54], [0.00451, 3.43], [0.0045, 10.06], [0.00449, 3.44], [0.00448, 0.7], [0.00447, 3.68], [0.00446, 1.0], [0.00445, 5.32], [0.00444, 0.72], [0.00443, 3.13], [0.00442, 2.26], [0.00441, 3.87], [0.0044, 1.0], [0.00439, 3.16], [0.00438, 0.36], [0.00437, 3.9], [0.00436, 0.36], [0.00435, 3.56], [0.00434, 1.49], [0.00433, 0.72], [0.00432, 0.8], [0.00431, 0.37], [0.0043, 0.37], [0.00429, 0.91], [0.00428, 0.74], [0.00427, 10.61], [0.00425, 0.99], [0.00424, 2.23], [0.00422, 0.37], [0.00421, 0.38], [0.0042, 0.76], [0.00418, 0.76], [0.00417, 0.38], [0.00416, 1.38], [

In [4]:
from typing import List, Tuple

async def _generate_triangular_paths(exchange) -> List[Tuple[str, str, str, str, str, str]]:
    """Generate triangular arbitrage paths from active spot markets with optional asset filtering."""
    try:
        markets = await exchange.load_markets()
        
        # Step 1: Filter only valid spot markets
        spot_markets = {
            symbol: market for symbol, market in markets.items()
            if market.get('spot') and market.get('active') and '/' in symbol
        }
        
        print(f"[{exchange.id}] Found {len(spot_markets)} active spot markets.")

        # Step 2: Build asset-to-pairs map
        asset_pairs = {}
        for symbol in spot_markets:
            base, quote = symbol.split('/')
            for asset in [base, quote]:
                asset_pairs.setdefault(asset, []).append(symbol)

        # Step 3: Find all valid triangular paths
        valid_paths = set()
        for pair1 in spot_markets:
            asset1, asset2 = pair1.split('/')
            
            for pair2 in asset_pairs.get(asset2, []):
                if pair2 == pair1:
                    continue
                
                base2, quote2 = pair2.split('/')
                asset3 = base2 if base2 != asset2 else quote2

                # Look for closing pair that completes the cycle
                pair3_options = [f"{asset3}/{asset1}", f"{asset1}/{asset3}"]
                for pair3 in pair3_options:
                    if pair3 in spot_markets:
                        valid_paths.add((asset1, asset2, asset3, pair1, pair2, pair3))
                        break

        print(f"[{exchange.id}] Generated {len(valid_paths)} total triangular paths.")

        # Step 4 (Optional): Filter paths involving only high-liquidity or priority assets
        priority_assets = {'USDT', 'BTC', 'ETH', 'BNB', 'USDC'}
        filtered_paths = [
            path for path in valid_paths
            if any(asset in priority_assets for asset in path[:3])
        ]

        print(f"[{exchange.id}] Filtered to {len(filtered_paths)} priority paths.")
        await exchange.close()
        return list(filtered_paths)

    except Exception as e:
        print(f"[{exchange.id}] Failed to generate triangular paths: {e}")
        return []


In [3]:
import asyncio
import ccxt.pro as cxt

exchange = cxt.binance({
        'apiKey': 'mb4MZz0XWG6rpNhYAtONez7AFnCjVv1gbq6oDBOb01p2QHQb6W3s6pSiLhNPJ1vl',
        'secret': '1FPwaxy4K52mOtV2y4iJ3nhgXmocccqlojllXvMfXnQ5Q8PGxgt108rSRd67UFds',
        'options': {'defaultType': 'spot'},
        'enableRateLimit': True,
        'sandbox': False,  # Set to True for testing
    })

res = await exchange.create_market_buy_order_with_cost('ETH/USDT', 10, {'test': True})
print(res)
await exchange.close()

{'info': {}, 'id': None, 'clientOrderId': None, 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'ETH/USDT', 'type': None, 'timeInForce': None, 'postOnly': False, 'reduceOnly': None, 'side': None, 'price': None, 'triggerPrice': None, 'amount': None, 'cost': None, 'average': None, 'filled': 0.0, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'fees': [], 'stopPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None}


In [ ]:
import asyncio
import ccxt.pro as cxt

exchange = cxt.bybit({
            'apiKey': 'rXK4kRziFR25WtwBgN',
            'secret': 'hTV6liP2e7afHsgWn5uz0v5N3Tk21jCnJbxn',
            'options': {'defaultType': 'spot'},
            'enableRateLimit': True,
            'sandbox': False,  # Set to True for testing
        })


res = await exchange.fetch_deposit_address('USDT', {'network': 'APT'})

print(res)
await exchange.close()

{'info': {'chainType': 'APTOS', 'addressDeposit': '0x0fe8cefe48e8439d21a58e2c0f96c5f88c759e8381b3a0009c662a31a0d0d62e', 'tagDeposit': '', 'chain': 'APTOS', 'batchReleaseLimit': '-1', 'contractAddress': 'c9dc2b'}, 'currency': 'USDT', 'network': 'APT', 'address': '0x0fe8cefe48e8439d21a58e2c0f96c5f88c759e8381b3a0009c662a31a0d0d62e', 'tag': None}


In [9]:
seen = [{'pair':'ETH/USDT', 'profit': '2'}]
op1 = {'pair':'ETH/USDT', 'profit': '2', 'pcnt':None}
op2 = {'pair':'BTC/USDT', 'profit': '2'}
print(op1 in seen)

False


In [7]:
class Base:
    account_balance = {}  # class-level

    def __init__(self):
        pass  # instance-specific

    def update_balance(self, asset, amount):
        self.account_balance.update({asset: amount})

class A(Base):
    def __init__(self):
        super().__init__()

    def main(self, amount):
        self.update_balance('A', amount)
    

class B(Base):
    def __init__(self):
        super().__init__()
    

a = A()
b = B()

# a.account_balance['A'] = 100
a.main(100)
print(a.account_balance)  # {'A': 100}
print(b.account_balance)  # {'A': 100}
a.main(200)
print(a.account_balance)  # {'A': 200}
print(b.account_balance)  # {'A': 200}


{'A': 100}
{'A': 100}
{'A': 200}
{'A': 200}
